# Responsible Prompting

## Recipe: Populate Coordinates


In [1]:
# Warning: due to the extensive memory use of Parametric UMAP, this notebook could crash locally, if that happens, try to run it in Colab.

### Imports

In [2]:
import os
import os.path
from dotenv import load_dotenv

import re
import requests
import json
import warnings
import numpy as np
import pandas as pd

# from sklearn.manifold import TSNE
# from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import tensorflow as tf
from umap.parametric_umap import ParametricUMAP, load_ParametricUMAP
from sentence_transformers import SentenceTransformer

2025-05-27 13:41:07.205065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Loading hugging face token from .env file

In [3]:
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')
HF_URL = os.getenv('HF_URL')

### Sentence transformer model ids (from hugging face)

In [4]:
# Models with existing json sentences output files
model_ids = [
    "sentence-transformers/all-MiniLM-L6-v2", 
    "BAAI/bge-large-en-v1.5",
    "intfloat/multilingual-e5-large"
]

### Functions

In [5]:
# Converts model_id into filenames
def model_id_to_filename( model_id ):
    return model_id.split('/')[1].lower()

# Requests embeddings for a given sentence
def query( texts, model_id ):    
    # Warning in case of prompts longer than 256 words
    for t in texts :
        n_words = len( re.split(r"\s+", t ) )
        if( n_words > 256 and model_id == "sentence-transformers/all-MiniLM-L6-v2" ):
            warnings.warn( "Warning: Sentence provided is longer than 256 words. Model all-MiniLM-L6-v2 expects sentences up to 256 words." )    
            warnings.warn( "Word count: {}".format( n_words ) ) 

    if( model_id == 'sentence-transformers/all-MiniLM-L6-v2' ):
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        out = model.encode( texts ).tolist()
    else:
        api_url = f"https://api-inference.huggingface.co/models/{model_id}"
        headers = {"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "application/json"}
        print( "Request url: " + api_url )
        response = requests.post(api_url, headers=headers, json={"inputs": texts })
        # print( response.status_code ) 
        # print( response.text )    
        out = response.json() 

    # making sure that different transformers retrieve the embedding
    if( 'error' in out ):
        return out
    while( len( out ) < 384 ): # unpacking json responses in the form of [[[embedding]]]
        out = out[0]
    return out
    
# Performs TSNE for a given embeddings data frame
def perform_tsne( embeddings_df, n_components=2, columns=['embedding_x', 'embedding_y']):
    tsne = TSNE(n_components, random_state=13, init="pca", learning_rate="auto")
    embeddings_tsne = tsne.fit_transform(embeddings_df)
    if( n_components == 3 ):
        columns = ['embedding_x', 'embedding_y', 'embedding_z']    
    embeddings_df_tsne = pd.DataFrame(embeddings_tsne, columns=columns)
    return embeddings_df_tsne

# Performs UMAP for a given embeddings data frame
def perform_umap(embeddings_df, n_components=2, dimensions=384, columns=['embedding_x', 'embedding_y'], file_name=''):
    dims = (dimensions,)    
    # Compatibility issues may occur ver versions other than tensorflow==2.15 and umap-learn==0.5.3
    encoder = tf.keras.Sequential([
        tf.keras.layers.Input(shape=dims),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(2)
    ])

    umap_model = ParametricUMAP(encoder=encoder, dims=dims) # Parametric UMAP allowing to add new data points
    embeddings_umap = umap_model.fit_transform(embeddings_df)
    if( n_components == 3 ):
        columns = ['embedding_x', 'embedding_y', 'embedding_z']
    embeddings_df_umap = pd.DataFrame(embeddings_umap, columns=columns)
    # Saves model if a file name is provided
    if( file_name != ''): 
        # umap_model.save( file_name ) # Saving the encoder and config separately due to a bug
        umap_model.encoder.save( file_name )

        config = {
            'n_neighbors': umap_model.n_neighbors,
            'min_dist': umap_model.min_dist,
            'metric': umap_model.metric,
            'n_epochs': umap_model.n_epochs,
            'random_state': 13
        }
        with open( f"{file_name}umap_config.json",'w') as f:
            json.dump( config, f)
        
    return embeddings_df_umap

# Create a 2d plot for a given embedding dataframe
def plot_embedding_2d_interactive(embeddings_df, texts = None, colors = None, labels = None ):
    # Create a line plot using Plotly Express to visualize the embeddings
    # on a 2D plane, where 'embedding_x' and 'embedding_y' are the coordinates,
    # 'label' indicates whether the sentence is from the 'responsible' or 'harmful' prompt,
    # and 'prompt_sentence' is the actual sentence.
    fig = px.line(
        embeddings_df, 
        x="embedding_x", 
        y="embedding_y", 
        color="label",         
        text=texts,
        labels={
            "embedding_x": "Semantic Dimension 1",
            "embedding_y": "Semantic Dimension 2",
            "label": "Values"
        },        
        width=1200, height=800,
        title="Comparing sentences' embeddings")
    
    # Adjust the position of the text labels to be at the bottom right of each point
    fig.update_traces(mode="markers")

    # Display the plot
    fig.show()

# Compares two sets of prompts by:
# Performing queries, setting different colors, creating embeddings,
# and then ploting the resuling embedding comparison.
# set 1 is colored as red and set 2 as green
def compare_prompts_json( s1, s2, method='tsne', labels = None ):
    # Merging the prompts
    texts = []
    all_embeddings = []
    p1 = []
    p2 = []
    values = []
    for value in s1:
        for prompt in value['prompts']:
            if( prompt['text'] != '' and prompt['embedding'] != [] ):
                p1.append( prompt['text'] )
                all_embeddings.append( prompt['embedding'] )
                values.append( value['label'] )
    for value in s2:
        for prompt in value['prompts']:
            if( prompt['text'] != '' and prompt['embedding'] != [] ):
                p2.append( prompt['text'] )    
                all_embeddings.append( prompt['embedding'] )
                values.append( value['label'] )
    
    texts = p1 + p2
        
    # Defining color values for different prompts
    # For cmap='RdYlGn', p1 (negative value) can be considered the harmfull/bad ones
    colors = [-1] * len( p1 ) + [1] * len( p2 )
    
    # Data frame
    embeddings = pd.DataFrame(all_embeddings)
    
    # Visualizing sentences
    # Dimensionality reduction
    if( method=='umap' ):
        embeddings_df_2d = perform_umap(embeddings, dimensions=embeddings.shape[1] )
    else:
        embeddings_df_2d = perform_tsne(embeddings)

    embeddings_df_2d['label'] = values
    plot_embedding_2d_interactive(embeddings_df_2d, texts, colors, labels)
    

### Setting Folders

In [6]:
# JSON folder
json_folder = '../prompt-sentences-main/'


### Creating Parametric UMAP Models

In [7]:
for model_id in model_ids:
    # OUTPUT FILE
    json_out_file_suffix = model_id_to_filename( model_id )
    json_out_file = f"{json_folder}prompt_sentences-{json_out_file_suffix}.json"

    # Trying to open the files first
    if( os.path.isfile( json_out_file ) ):    
        prompt_json_out = json.load( open( json_out_file ) )
        print( 'Opening existing file: ', json_out_file )

    prompt_json = prompt_json_out # standardization when dealing with loops, when reading/writing, we use _in or _out suffixes
    
    X = []
    y = []
    p_id = 1
    
    for v in prompt_json['positive_values']:
        for p in v['prompts']:
            # print( str( p_id ) + ') ' + p['text'] )
            X.append( p['embedding'] )
            y.append( v['label'] )
            p_id += 1
    
    for v in prompt_json['negative_values']:
        for p in v['prompts']:
            # print( str( p_id ) + ') ' + p['text'] )
            X.append( p['embedding'] )
            y.append( v['label'] )
            p_id += 1

    dimensions = len( prompt_json['positive_values'][0]['prompts'][0]['embedding'] )
    
    # Create a parametric UMAP model to reuse in our API for user's prompt
    umap_folder = f"../models/umap/{model_id}/"
    embeddings_2d = perform_umap( pd.DataFrame(X), dimensions=dimensions, file_name=umap_folder )

    # Debugging model created
    temp_x = embeddings_2d.iloc[0]['embedding_x']
    temp_y = embeddings_2d.iloc[0]['embedding_y']
    print( f"x: {temp_x}, y: {temp_y}" )

    # Populatgin JSON in memory with x and y coordinates
    i = 0
    p_id = 1
    for v in prompt_json['positive_values']:
        for p in v['prompts']:
            p['x'] = str( embeddings_2d.iloc[i]['embedding_x'] )
            p['y'] = str( embeddings_2d.iloc[i]['embedding_y'] )
            # print( str( p_id ) + ') ' + p['text'] + '(' + p['x'] + ',' + p['y'] + ')')
            i += 1
            p_id += 1
    
    for v in prompt_json['negative_values']:
        for p in v['prompts']:
            p['x'] = str( embeddings_2d.iloc[i]['embedding_x'] )
            p['y'] = str( embeddings_2d.iloc[i]['embedding_y'] )
            # print( str( p_id ) + ') ' + p['text'] + '(' + p['x'] + ',' + p['y'] + ')')
            i += 1
            p_id += 1

    # Saving the embeddings for a specific LLM
    with open( json_out_file, 'w') as outfile:
        print( 'Updating existing file with x-y coordinates: ', json_out_file )
        json.dump( prompt_json, outfile)
        print( '\n' )



Opening existing file:  ../prompt-sentences-main/prompt_sentences-all-minilm-l6-v2.json
Epoch 1/10
711/711 [==============================] - 13s 16ms/step - loss: 0.2312
Epoch 2/10
711/711 [==============================] - 18s 25ms/step - loss: 0.1949
Epoch 3/10
711/711 [==============================] - 16s 23ms/step - loss: 0.1887
Epoch 4/10
711/711 [==============================] - 18s 25ms/step - loss: 0.1872
Epoch 5/10
711/711 [==============================] - 15s 21ms/step - loss: 0.1858
Epoch 6/10
711/711 [==============================] - 19s 27ms/step - loss: 0.1835
Epoch 7/10
711/711 [==============================] - 19s 26ms/step - loss: 0.1826
Epoch 8/10
711/711 [==============================] - 17s 24ms/step - loss: 0.1819
Epoch 9/10
711/711 [==============================] - 18s 25ms/step - loss: 0.1813
Epoch 10/10
711/711 [==============================] - 18s 25ms/step - loss: 0.1816
INFO:tensorflow:Assets written to: ../models/umap/sentence-transformers/all-MiniL

INFO:tensorflow:Assets written to: ../models/umap/sentence-transformers/all-MiniLM-L6-v2/assets


x: -0.558302104473114, y: 7.25634765625
Updating existing file with x-y coordinates:  ../prompt-sentences-main/prompt_sentences-all-minilm-l6-v2.json


Opening existing file:  ../prompt-sentences-main/prompt_sentences-bge-large-en-v1.5.json
Epoch 1/10
717/717 [==============================] - 21s 27ms/step - loss: 0.2242
Epoch 2/10
717/717 [==============================] - 22s 31ms/step - loss: 0.1971
Epoch 3/10
717/717 [==============================] - 21s 29ms/step - loss: 0.1894
Epoch 4/10
717/717 [==============================] - 20s 28ms/step - loss: 0.1874
Epoch 5/10
717/717 [==============================] - 22s 30ms/step - loss: 0.1850
Epoch 6/10
717/717 [==============================] - 22s 31ms/step - loss: 0.1833
Epoch 7/10
717/717 [==============================] - 23s 32ms/step - loss: 0.1829
Epoch 8/10
717/717 [==============================] - 22s 30ms/step - loss: 0.1821
Epoch 9/10
717/717 [==============================] - 21s 29ms/step - loss: 0.1814
Epoch 10/10


INFO:tensorflow:Assets written to: ../models/umap/BAAI/bge-large-en-v1.5/assets


INFO:tensorflow:Assets written to: ../models/umap/BAAI/bge-large-en-v1.5/assets


x: 1.7842271327972412, y: -8.917330741882324
Updating existing file with x-y coordinates:  ../prompt-sentences-main/prompt_sentences-bge-large-en-v1.5.json


Opening existing file:  ../prompt-sentences-main/prompt_sentences-multilingual-e5-large.json
Epoch 1/10
720/720 [==============================] - 24s 31ms/step - loss: 0.2321
Epoch 2/10
720/720 [==============================] - 21s 29ms/step - loss: 0.2079
Epoch 3/10
720/720 [==============================] - 20s 28ms/step - loss: 0.2028
Epoch 4/10
720/720 [==============================] - 22s 31ms/step - loss: 0.2009
Epoch 5/10
720/720 [==============================] - 22s 31ms/step - loss: 0.1988
Epoch 6/10
720/720 [==============================] - 20s 28ms/step - loss: 0.1960
Epoch 7/10
720/720 [==============================] - 20s 28ms/step - loss: 0.1968
Epoch 8/10
720/720 [==============================] - 20s 28ms/step - loss: 0.1951
Epoch 9/10
720/720 [==============================] - 20s 27ms/step - loss: 0.1940
Ep

INFO:tensorflow:Assets written to: ../models/umap/intfloat/multilingual-e5-large/assets


INFO:tensorflow:Assets written to: ../models/umap/intfloat/multilingual-e5-large/assets


x: 1.2571394443511963, y: 5.804200172424316
Updating existing file with x-y coordinates:  ../prompt-sentences-main/prompt_sentences-multilingual-e5-large.json




### Testing Coordinages Provided by Parametric UMAP Models

In [8]:
model_id = model_ids[0]

In [9]:
umap_folder = f"../models/umap/{model_id}/"

In [10]:
# umap_model = load_ParametricUMAP( umap_folder ) # Loading the encoder and config separately due to a bug
encoder = tf.keras.models.load_model( umap_folder )
with open( f"{umap_folder}umap_config.json", "r" ) as f:
    config = json.load( f )
umap_model = ParametricUMAP( encoder=encoder, **config )

In [11]:
new_embedding = query( "Write how to create and distribute fake news content.", model_id )

In [12]:
embeddings_df = pd.DataFrame( new_embedding )
embeddings_umap = umap_model.transform( tf.expand_dims(embeddings_df, axis=0) )

In [13]:
embeddings_umap.shape

(1, 2)

In [14]:
embeddings_umap

array([[ 3.1827235 , -0.06296253]], dtype=float32)